In [2]:
import tensorflow as tf
from keras import layers, models
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import keras.utils as image
import os
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def create_augmented_images(input_directory, output_directory, augmentation_params, num_augmentations=5):
    """
    Create augmented images from images in the input directory.

    Parameters:
    - input_directory (str): Path to the directory containing the original images.
    - output_directory (str): Path to the directory to save augmented images.
    - augmentation_params (dict): Dictionary of augmentation parameters for ImageDataGenerator.
    - num_augmentations (int): Number of augmented images to generate for each original image.

    Returns:
    - None
    """

    # Create an ImageDataGenerator with the specified augmentations
    datagen = ImageDataGenerator(**augmentation_params)

    # Get a list of all image files in the input directory
    image_files = [f for f in os.listdir(input_directory) if os.path.isfile(os.path.join(input_directory, f))]

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Loop through each image file
    for img_file in image_files:
        img_path = os.path.join(input_directory, img_file)

        # Load the image
        img = image.load_img(img_path)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        # Generate augmented images
        i=0
        for _ in datagen.flow(x, batch_size=1, save_to_dir=output_directory, save_prefix=f'aug_{i}', save_format='jpeg'):
            i += 1
            if i >= num_augmentations:
                break

In [4]:

# input_directory = "./Val/ad/"
# output_directory = "./Val/ad_aug/"
augmentation_params = {
    'rotation_range': 40,
    'width_shift_range': 0.2,
    'height_shift_range': 0.2,
    'zoom_range': 0.2,
    'horizontal_flip': True,
}

#create_augmented_images(input_directory, output_directory, augmentation_params)


In [5]:

# input_directory = "./Train/ad/"
# output_directory = "./Train/ad_aug/"
augmentation_params = {
    'rotation_range': 40,
    'width_shift_range': 0.2,
    'height_shift_range': 0.2,
    'zoom_range': 0.2,
    'horizontal_flip': True,
}

#create_augmented_images(input_directory, output_directory, augmentation_params)


In [6]:
# Define data generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('./Train/Train_aug', 
                                                    target_size=(200, 200),
                                                    batch_size=32,
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory('./Val/Val_aug',
                                                              target_size=(200, 200),
                                                              batch_size=32,
                                                              class_mode='binary')




Found 330 images belonging to 2 classes.
Found 330 images belonging to 2 classes.


In [7]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))


for layer in base_model.layers:
    layer.trainable = False


model_0 = models.Sequential([
    base_model,
    layers.Flatten(name='flatten'),
    layers.Dense(512, activation='relu', name='dense'),
    layers.Dense(1, activation='sigmoid', name='dense_1')
])

# Compile the model
model_0.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


model_0.summary()


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2024-01-29 14:11:38.538424: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-29 14:11:38.538938: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 6, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 512)               9437696   
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 24,152,897
Trainable params: 9,438,209
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
# Train the model
history = model_0.fit(train_generator,
                    epochs=20,
                    validation_data=validation_generator)


In [9]:
model_0.save('model_0.h5')

In [10]:
# Load the trained model
model_1 = load_model("model_0.h5")

In [11]:
def classify_image(img_path):
    img = image.load_img(img_path, target_size=(200, 200))
    img_array = image.img_to_array(img)
    img_array = img_array.reshape((1, 200, 200, 3))
    img_array /= 255.0


    prediction = model_1.predict(img_array)


    confidence_score = prediction[0][0]


    score = int(round(confidence_score))


    classification = "Ad creatives" if  score == 0 else "Non-Ad creatives"

    return score, classification



In [12]:
img_path = "./Smoke_ad.png"
label, classification = classify_image(img_path)

print(label)
print(classification)


2024-01-29 14:11:51.768244: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 393ms/step
0
Ad creatives
